## Init

In [35]:
import argparse
import os.path as osp
import xml.etree.ElementTree as ET
from pandas import read_csv
import mmcv
import numpy as np
from PIL import Image
import pickle

In [2]:
!pwd

/home/nikstar/mmdetection


In [56]:
root = '.' # mmdetection root directory

## Convert dataset from csv to COCO format


In [121]:
dataset_dir = osp.join(root, 'data/rtsd')
split = ['train', 'test']
csv_paths = [osp.join(dataset_dir, s + '-rtsd.csv') for s in split]
images_dir = osp.join(dataset_dir, 'rtsd-frames')
csv_paths, images_dir

(['./data/rtsd/train-rtsd.csv', './data/rtsd/test-rtsd.csv'],
 './data/rtsd/rtsd-frames')

#### Collect classes:

In [140]:
classes = set()
classes_dict = dict()

for path in csv_paths:
    df = read_csv(path)
    for idx, row in df.iterrows():
        classes.add(row.sign_class)
    
classes = list(classes)
classes.sort()

for idx, cl in enumerate(classes):
    classes_dict[cl] = idx

print(classes)

['1.1', '1.10', '1.11.1', '1.11.2', '1.12.1', '1.12.2', '1.13', '1.14', '1.15', '1.16', '1.17', '1.18', '1.19', '1.2', '1.20.1', '1.20.2', '1.20.3', '1.21', '1.22', '1.23', '1.25', '1.26', '1.27', '1.30', '1.31', '1.33', '1.5', '1.6', '1.7', '1.8', '2.1', '2.2', '2.3.1', '2.3.2', '2.3.3', '2.3.4', '2.3.5', '2.3.6', '2.4', '2.5', '2.6', '2.7', '3.1', '3.10', '3.11.n13', '3.11.n17', '3.11.n20', '3.11.n23', '3.11.n5', '3.11.n8', '3.11.n9', '3.12.n10', '3.12.n3', '3.12.n5', '3.12.n6', '3.13.r', '3.13.r2.5', '3.13.r2.6', '3.13.r3', '3.13.r3.3', '3.13.r3.5', '3.13.r3.7', '3.13.r3.9', '3.13.r4.0', '3.13.r4.1', '3.13.r4.2', '3.13.r4.3', '3.13.r4.5', '3.13.r5', '3.13.r5.2', '3.14.r2.7', '3.14.r3', '3.14.r3.5', '3.14.r3.7', '3.16.n', '3.16.n1', '3.16.n3', '3.18.1', '3.18.2', '3.19', '3.2', '3.20', '3.21', '3.24.n', '3.24.n10', '3.24.n20', '3.24.n30', '3.24.n40', '3.24.n45', '3.24.n5', '3.24.n50', '3.24.n60', '3.24.n70', '3.24.n80', '3.25.n', '3.25.n20', '3.25.n40', '3.25.n50', '3.25.n70', '3.25.

#### Parse and dump into pickle files:

In [120]:
%%time
for path in csv_paths:
    print('processing {} ...'.format(path))
    dataset = parse_csv(path)
    output_filename = path[:-4] + '-TESTING.pkl'
#     output_filename = path[:-4] + '.pkl'
    with open(output_filename, 'wb') as f:
        pickle.dump(dataset, f)
print('Done!')

processing ./data/rtsd/train-rtsd.csv ...
processing ./data/rtsd/test-rtsd.csv ...
Done!
CPU times: user 1min 47s, sys: 598 ms, total: 1min 48s
Wall time: 1min 48s


In [118]:
%%time

def parse_csv(path, limit=None):
    df = read_csv(path)
    if limit is not None:
        df = df.head(limit)
    
    # collect unique filenames
    filenames = list(set(r.filename for _, r in df.iterrows()))
    filenames.sort()
    
    # parse annotations
    annotations_dict = dict()
    for idx, row in df.iterrows():
        
        filename = row.filename
        
        if annotations_dict.get(filename) is None:
            if row.ignore == 'True':
                print(row)
                break
            row = load_image(row)
            bbox, label = parse_ann(row)
            annotations_dict[filename] = {
               'filename': 'rtsd-frames/' + filename,
                'width': row.img_width,
                'height': row.img_height,
                'ann': {
                    'bboxes': np.array([bbox], dtype=np.float32),
                    'labels': np.array([label], dtype=np.int64),
                }
            }
        else:
            bbox, label = parse_ann(row)
            annotations_dict[filename]['ann']['bboxes'] = np.append(annotations_dict[filename]['ann']['bboxes'], np.array([bbox], dtype=np.float32), axis=0)
            annotations_dict[filename]['ann']['labels'] = np.append(annotations_dict[filename]['ann']['labels'], np.array([label], dtype=np.int64), axis=0)
    
    # return in correct order
    annotations = []
    for filename in filenames:
        annotations.append(annotations_dict[filename])
    
    return annotations

def load_image(row):
    img_name = row['filename']
    img_path = osp.join(images_dir, img_name)
    row['img_width'], row['img_height'] = Image.open(img_path).size
    return row


def parse_ann(row):
    bbox = [
        int(row['x']), int(row['y']),
        int(row['x']) + int(row['w']), int(row['y']) + int(row['h']),
    ]
    label = classes_dict[row.sign_class]
    return bbox, label

dataset = parse_csv(csv_paths[1], limit=10)
dataset[-1]

CPU times: user 42.7 ms, sys: 0 ns, total: 42.7 ms
Wall time: 42.8 ms


{'filename': 'rtsd-frames/autosave01_02_2012_09_15_20.jpg',
 'width': 1280,
 'height': 720,
 'ann': {'bboxes': array([[799., 310., 835., 348.],
         [718., 355., 738., 380.]], dtype=float32),
  'labels': array([140, 137])}}

# Explore RTSD dataset

In [133]:
with open(osp.join(dataset_dir, 'train-rtsd.pkl'), 'rb') as f:
    dataset = pickle.load(f)
dataset[-1]

{'filename': 'rtsd-frames/autosave24_10_2013_13_18_54_3.jpg',
 'width': 1280,
 'height': 720,
 'ann': {'bboxes': array([[910., 309., 972., 363.]], dtype=float32),
  'labels': array([18])}}

In [136]:
with open(osp.join(dataset_dir, 'test-rtsd.pkl'), 'rb') as f:
    dataset = pickle.load(f)
dataset[-1]

{'filename': 'rtsd-frames/autosave24_10_2013_13_18_54_1.jpg',
 'width': 1280,
 'height': 720,
 'ann': {'bboxes': array([[1083.,  241., 1169.,  317.]], dtype=float32),
  'labels': array([135])}}

In [137]:
count = len(dataset)
files = set(d['filename'] for d in dataset)
labels = list(d['ann']['labels'].size for d in dataset)
len(files), sum(labels)

(11650, 25613)

In [138]:
from collections import Counter
dict(Counter(labels))

{1: 4442,
 2: 3546,
 3: 1907,
 4: 985,
 5: 445,
 8: 39,
 7: 79,
 6: 182,
 10: 9,
 9: 15,
 11: 1}

In [105]:
count_labels = { d['filename']: d['ann']['labels'].size for d in dataset}

{'rtsd-frames/autosave01_02_2012_09_13_33.jpg': 1,
 'rtsd-frames/autosave01_02_2012_09_13_34.jpg': 1,
 'rtsd-frames/autosave01_02_2012_09_13_35.jpg': 1,
 'rtsd-frames/autosave01_02_2012_09_13_36.jpg': 2,
 'rtsd-frames/autosave01_02_2012_09_13_37.jpg': 2,
 'rtsd-frames/autosave01_02_2012_09_13_38.jpg': 1,
 'rtsd-frames/autosave01_02_2012_09_13_39.jpg': 1,
 'rtsd-frames/autosave01_02_2012_09_13_42.jpg': 2,
 'rtsd-frames/autosave01_02_2012_09_13_43.jpg': 3,
 'rtsd-frames/autosave01_02_2012_09_13_44.jpg': 4,
 'rtsd-frames/autosave01_02_2012_09_13_45.jpg': 3,
 'rtsd-frames/autosave01_02_2012_09_13_47.jpg': 1,
 'rtsd-frames/autosave01_02_2012_09_13_48.jpg': 2,
 'rtsd-frames/autosave01_02_2012_09_13_49.jpg': 3,
 'rtsd-frames/autosave01_02_2012_09_13_50.jpg': 3,
 'rtsd-frames/autosave01_02_2012_09_13_51.jpg': 2,
 'rtsd-frames/autosave01_02_2012_09_13_54.jpg': 3,
 'rtsd-frames/autosave01_02_2012_09_13_55.jpg': 3,
 'rtsd-frames/autosave01_02_2012_09_13_57.jpg': 2,
 'rtsd-frames/autosave01_02_201

# Explore VOC dataset format

In [122]:
voc_path = osp.join(root, 'data/VOCdevkit')
path = osp.join(voc_path, 'voc12_train.pkl')
with open(path, 'rb') as f:
    data = pickle.load(f)
    print(data[-1])

{'filename': 'VOC2012/JPEGImages/2011_003276.jpg', 'width': 500, 'height': 400, 'ann': {'bboxes': array([[263., 141., 291., 216.]], dtype=float32), 'labels': array([3]), 'bboxes_ignore': array([], shape=(0, 4), dtype=float32), 'labels_ignore': array([], dtype=int64)}}


In [87]:
csv_path = './rtsd'
img_path = csv_path + '/rtsd-frames'
out_dir = '.'

for split in ['train', 'test']:
    dataset_name = 'rtsd_' + split 
    print('processing {} ...'.format(dataset_name))

    csv_filename = csv_path + '/' + split + '-rtsd.csv'
    parse_csv(csv_filename, img_path, osp.join(out_dir, dataset_name + '.pkl'))

    print('Done!')

processing rtsd_train ...
Done!
processing rtsd_test ...
Done!
